In [1]:
import pandas as pd
import re
from datetime import timedelta
import sys
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime
from textblob import TextBlob

In [47]:

df=pd.read_excel("Lysol SS 28th July - 3rd Aug.xlsx")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           778 non-null    int64         
 1   ARTICLE_ID                   778 non-null    int64         
 2   EXTERNAL_ID                  778 non-null    object        
 3   EXTERNAL_AUTHOR_ID           773 non-null    object        
 4   HEADLINE                     778 non-null    object        
 5   AUTHOR                       778 non-null    object        
 6   Full Text                    766 non-null    object        
 7   ARTICLE_URL                  778 non-null    object        
 8   MEDIA_PROVIDER               778 non-null    object        
 9   REGION                       778 non-null    object        
 10  LANGUAGE                     778 non-null    object        
 11  POST_STATUS                  778 non-null    

In [48]:
#df['Combined Text']=df['Title']+" [$] "+df['Review']

In [49]:
df.columns

Index(['ID', 'ARTICLE_ID', 'EXTERNAL_ID', 'EXTERNAL_AUTHOR_ID', 'HEADLINE',
       'AUTHOR', 'Full Text', 'ARTICLE_URL', 'MEDIA_PROVIDER', 'REGION',
       'LANGUAGE', 'POST_STATUS', 'PUBLISH_DATE', 'HARVESTED_DATE',
       'SENTIMENT', 'POST_TYPE', 'LIKES_AND_VOTES', 'ENGAGEMENT',
       'UNIQUE_COMMENTERS', 'COMMENT_COUNT', 'WORKFLOW_ENGAGEMENT',
       'WORKFLOW_CLASSIFICATION', 'WORKFLOW_ASSIGNMENT', 'WORKFLOW_PRIORITY',
       'WORKFLOW_BLOGPOSTNOTES', 'WORKFLOW_BLOGPOSTTAGS', 'WORKFLOW_BLOGNOTES',
       'WORKFLOW_BLOGTAGS', 'WORKFLOW_AUTHORNOTES', 'WORKFLOW_AUTHORTAGS',
       'FIRST_ENGAGEMENT_ACTIVITY', 'LAST_ENGAGEMENT_ACTIVITY',
       'DURATION_PUBLISHED_TO_FIRST', 'DURATION_FIRST_TO_LAST',
       'FIRST_ASSIGNMENT_DATE', 'Translated Text'],
      dtype='object')

In [50]:
#pip install afinn

In [51]:
from textblob import TextBlob

In [52]:
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

In [53]:
df['sentiment_polarity1'] = df['Translated Text'].apply(sentiment_calc)

In [54]:
sentiment_list = []
for i in df['sentiment_polarity1']:
    if i > 0:
        sentiment_list.append('Positive')
    elif i < 0:
        sentiment_list.append('Negative')
    else:
        sentiment_list.append('Neutral')
df['sentiment_t']=sentiment_list
        

In [55]:
pd.crosstab(index=df["sentiment_t"],columns="count") 

col_0        count
sentiment_t       
Negative        41
Neutral        206
Positive       531

In [56]:
#!pip install afinn

In [57]:
from afinn import Afinn
af = Afinn()

In [58]:
def sentiment_calc1(text):
    try:
        return af.score(text)
    except:
        return None

In [59]:
df['sentiment_polarity2'] = df['Translated Text'].apply(sentiment_calc1)

In [60]:
sentiment_list = []
for i in df['sentiment_polarity2']:
    if i > 0:
        sentiment_list.append('Positive')
    elif i < 0:
        sentiment_list.append('Negative')
    else:
        sentiment_list.append('Neutral')
df['sentiment_a']=sentiment_list
        

In [61]:
pd.crosstab(index=df["sentiment_a"],columns="count") 

col_0        count
sentiment_a       
Negative        32
Neutral        161
Positive       585

In [62]:
#!pip install vaderSentiment

In [63]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [64]:
sid_obj = SentimentIntensityAnalyzer()

In [65]:
def sentiment_scores(sentence):
    try:
        return sid_obj.polarity_scores(sentence) 
    except:
        return None

In [66]:
sentiment_dict = df['Translated Text'].apply(sentiment_scores)

In [67]:
#df1=sentiment_dict.to_dict()

In [68]:
df["senti_dict"]=sentiment_dict

In [69]:
sentiment_dict[0]["compound"]

0.0

In [70]:
a=[]
for i in range(len(df["senti_dict"])):
    try:
        a.append(df["senti_dict"][i]['compound'])
    except:
        a.append("NA")


In [71]:
df["van_score"]=a

In [72]:
df.head(1)

ID     ARTICLE_ID                                      EXTERNAL_ID  \
0   1  1636342072558  117877892463125_828302024754038_715627516202365   

  EXTERNAL_AUTHOR_ID                    HEADLINE        AUTHOR   Full Text  \
0   5870947892916570  Comment From: Stacey Laker  STACEY LAKER  Josh Laker   

                                         ARTICLE_URL  \
0  http://www.facebook.com/permalink.php?story_fb...   

                  MEDIA_PROVIDER     REGION  ... DURATION_PUBLISHED_TO_FIRST  \
0  Facebook Unpublished Comments  Australia  ...                    00:01:03   

  DURATION_FIRST_TO_LAST FIRST_ASSIGNMENT_DATE Translated Text  \
0               00:00:00                   NaT      Josh Laker   

  sentiment_polarity1  sentiment_t  sentiment_polarity2  sentiment_a  \
0                 0.0      Neutral                  0.0      Neutral   

                                          senti_dict  van_score  
0  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...        0.0  

[1 rows x 42 columns]

In [73]:
s_list=[]
for i in df["van_score"]:
    try:
        if i > 0.1:
            s_list.append('Positive')
        elif i < 0:
            s_list.append('Negative')
        else:
            s_list.append('Neutral')
    except:
            s_list.append('NA')

        

In [74]:
df['sentiment1']=s_list

In [75]:
pd.crosstab(index=df["sentiment1"],columns="count") 


col_0       count
sentiment1       
NA             12
Negative       28
Neutral       146
Positive      592

In [76]:
df['Analytics Sentiment']=df[['sentiment1','sentiment_a', 'sentiment_t']].mode(axis=1)[0]

In [77]:
pd.crosstab(index=df['Analytics Sentiment'],columns="count") 

col_0                count
Analytics Sentiment       
Negative                32
Neutral                158
Positive               588

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           778 non-null    int64         
 1   ARTICLE_ID                   778 non-null    int64         
 2   EXTERNAL_ID                  778 non-null    object        
 3   EXTERNAL_AUTHOR_ID           773 non-null    object        
 4   HEADLINE                     778 non-null    object        
 5   AUTHOR                       778 non-null    object        
 6   Full Text                    766 non-null    object        
 7   ARTICLE_URL                  778 non-null    object        
 8   MEDIA_PROVIDER               778 non-null    object        
 9   REGION                       778 non-null    object        
 10  LANGUAGE                     778 non-null    object        
 11  POST_STATUS                  778 non-null    

In [79]:
a=df.columns
a

Index(['ID', 'ARTICLE_ID', 'EXTERNAL_ID', 'EXTERNAL_AUTHOR_ID', 'HEADLINE',
       'AUTHOR', 'Full Text', 'ARTICLE_URL', 'MEDIA_PROVIDER', 'REGION',
       'LANGUAGE', 'POST_STATUS', 'PUBLISH_DATE', 'HARVESTED_DATE',
       'SENTIMENT', 'POST_TYPE', 'LIKES_AND_VOTES', 'ENGAGEMENT',
       'UNIQUE_COMMENTERS', 'COMMENT_COUNT', 'WORKFLOW_ENGAGEMENT',
       'WORKFLOW_CLASSIFICATION', 'WORKFLOW_ASSIGNMENT', 'WORKFLOW_PRIORITY',
       'WORKFLOW_BLOGPOSTNOTES', 'WORKFLOW_BLOGPOSTTAGS', 'WORKFLOW_BLOGNOTES',
       'WORKFLOW_BLOGTAGS', 'WORKFLOW_AUTHORNOTES', 'WORKFLOW_AUTHORTAGS',
       'FIRST_ENGAGEMENT_ACTIVITY', 'LAST_ENGAGEMENT_ACTIVITY',
       'DURATION_PUBLISHED_TO_FIRST', 'DURATION_FIRST_TO_LAST',
       'FIRST_ASSIGNMENT_DATE', 'Translated Text', 'sentiment_polarity1',
       'sentiment_t', 'sentiment_polarity2', 'sentiment_a', 'senti_dict',
       'van_score', 'sentiment1', 'Analytics Sentiment'],
      dtype='object')

In [80]:
df1=df.drop(['sentiment_polarity1', 'sentiment_t',
       'sentiment_polarity2', 'sentiment_a', 'senti_dict', 'van_score',
       'sentiment1'], axis = 1) 

In [81]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 37 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           778 non-null    int64         
 1   ARTICLE_ID                   778 non-null    int64         
 2   EXTERNAL_ID                  778 non-null    object        
 3   EXTERNAL_AUTHOR_ID           773 non-null    object        
 4   HEADLINE                     778 non-null    object        
 5   AUTHOR                       778 non-null    object        
 6   Full Text                    766 non-null    object        
 7   ARTICLE_URL                  778 non-null    object        
 8   MEDIA_PROVIDER               778 non-null    object        
 9   REGION                       778 non-null    object        
 10  LANGUAGE                     778 non-null    object        
 11  POST_STATUS                  778 non-null    

In [82]:
writer = pd.ExcelWriter(r"Lysol SS 28th July - 3rd Aug-senti.xlsx", engine='xlsxwriter',options={'strings_to_urls': False})
df1.to_excel(writer)
writer.close()

C:\Users\TRUPTI~1.RAU\AppData\Local\Temp/ipykernel_7064/3379380263.py:1: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter(r"Lysol SS 28th July - 3rd Aug-senti.xlsx", engine='xlsxwriter',options={'strings_to_urls': False})
